In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

import nltk
from nltk.corpus import stopwords

from IPython.display import clear_output # to clear the large outputs

In [2]:
df = pd.read_csv("./features/embeddings_using_word2vec.csv")
df.head()

,File Number,Sentence Number,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,...,Feature 91,Feature 92,Feature 93,Feature 94,Feature 95,Feature 96,Feature 97,Feature 98,Feature 99,Feature 100
0,0,0,-0.168716,-5.406358,1.326604,9.634008,-0.076849,-1.104372,-0.415806,-0.480926,...,5.358544,-1.592642,1.034011,9.535279,2.598292,5.612475,-4.350582,-0.964401,-1.292501,3.694599
1,0,1,-1.333690,-1.051619,0.216564,6.122249,2.214809,-0.725307,-0.830446,3.878453,...,-0.961587,0.208162,2.276517,3.888922,4.042837,5.367699,-1.329080,-0.513146,1.002153,1.096816
2,0,2,-0.755151,-1.945825,1.523190,7.578938,0.057999,-2.244677,-0.843832,2.504396,...,4.913831,-0.246198,0.594669,6.016462,1.722590,5.567656,-3.447634,0.647401,-0.364287,2.557158
3,0,3,0.059533,-9.801075,-1.865499,18.202757,18.659924,-4.779880,4.589435,-12.752604,...,-13.412796,9.317993,5.314600,20.374498,6.672820,28.156467,21.488192,-17.480896,-16.550327,8.181047
4,0,4,9.340207,-35.000464,-3.669642,28.096300,4.814506,-28.647469,-3.292072,8.200159,...,-11.041115,5.776569,9.864917,38.569895,17.010353,32.720708,4.465261,-31.103429,-17.649040,12.265246


# Create graph_info tuple
It consists of the following
- __node_features__ : This is a [num_nodes, num_features] shaped NumPy array that includes the node features. In this dataset, the nodes are the papers, and the node_features are the word-presence binary vectors of each paper.
- __edges__: This is [num_edges, num_edges] NumPy array representing a sparse adjacency matrix of the links between the nodes. In this example, the links are the citations between the papers.
- __edge_weights__ (optional): This is a [num_edges] NumPy array that includes the edge weights, which quantify the relationships between nodes in the graph. In this example, there are no weights for the paper citations.


In [10]:
n_input_features = 100

# Implement a graph convolution layer

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
# from spektral.layers.convolutional import GraphConv

In [8]:
from tensorflow.keras.layers import Layer

class GraphConvolutionLayer(Layer):
    def __init__(self, units):
        super(GraphConvolutionLayer, self).__init__()
        self.units = units
    
    def build(self, input_shape):
        self.weight = self.add_weight("weight", (input_shape[-1], self.units), initializer="random_normal")

    def call(self, inputs, adjecency_matrix):
        adjecency_matrix = tf.convert_to_tensor(adjecency_matrix, dtype=tf.float32, name="adjecency_matrix")
        adjecency_matrix = tf.linalg.diag(tf.reduce_sum(adjecency_matrix, axis=-1)) @ adjecency_matrix
        output = tf.matmul(adjecency_matrix, tf.matmul(inputs, self.weight))
        return output

# Implement a graph neural network node classifier


In [12]:
# Define a GNN model
class GNNModel(Model):
    def __init__(self, n_input_features, n_output_features):
        super(GNNModel, self).__init__()
        
        # The input will have the shape [n_nodes, n_input_features]
        self.graph_conv1 = GraphConvolutionLayer(64)
        self.graph_conv2 = GraphConvolutionLayer(32)
        self.dense = Dense(n_output_features, activation='softmax')  # Assume a classification problem for the node features
        
    def call(self, inputs):
        # inputs is a list: [node_features, adjacency_matrix]
        x, a = inputs
        
        # Pass through GNN layers
        x = self.graph_conv1([x, a])
        x = self.graph_conv2([x, a])
        
        # A dense layer for output
        return self.dense(x)


# Create the GNN model
n_output_features = 2  # For instance, classifying nodes into 2 categories (include or exclude)
model = GNNModel(n_input_features, n_output_features)

# You can now compile and train the model using Keras functionalities.
# The input to the model during training will be a list: [node_features, adjacency_matrix].

In [17]:
model.build(n_input_features)

ValueError: Specified input shape is not one of the valid types. Please specify a batch input shape of type tuple or list of input shapes. User provided input type: <class 'int'>.

In [15]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

# Train the GNN model


# Examine the GNN model predictions